In [1]:
import os
import dill
import numpy as np

import matplotlib
# Set the matplotlib backend to 'agg' to prevent it from rendering images in the notebook
matplotlib.use('agg')
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../../../src')

from biae import BayesianIQAEResult

## 0 Load the results.

In [2]:
thetas = [45] #range(0, 91, 5)
epsilon_idx = 7   # 7 for the last epsilon value (1e-8)

In [3]:
# Load variables
all_results = []
for theta in thetas:
    file_path = os.path.join('../all_angle_results', f'results_{theta}.pkl')
    with open(file_path, 'rb') as f:
        data_loaded = dill.load(f)
        # append 1000 results
        all_results.extend(data_loaded['results_bayes'][epsilon_idx]['results'])
        del data_loaded

## 1 Compute the ratios

Function to compute ratios.

In [4]:
def get_ratios(iae_result):
    # Filter out and keep only the theta_intervals 
    # that correspond to the last occurrence of each unique power value in the powers list
    ## Find the indices of the last occurrence of each unique power value
    unique_powers = list(set(iae_result.powers))
    last_indices = [len(iae_result.powers) - 1 - iae_result.powers[::-1].index(power) for power in unique_powers]
    
    ## Sort the last_indices to preserve the order in powers
    last_indices.sort()
    
    ## Filter the theta_intervals using the last_indices
    filtered_theta_intervals = [iae_result.theta_intervals[i] for i in last_indices]
    
    # Calculate the half-lengths of the filtered intervals
    radius = [(np.arcsin(np.sqrt(interval[1])) - np.arcsin(np.sqrt(interval[0]))) / 2 for interval in filtered_theta_intervals]
    # Compute the ratios
    ratios = [(radius[i-1] / radius[i]) for i in range(1, len(radius))]
    
    return ratios

Compute all ratios.

In [5]:
all_ratios = []

for iae_result in all_results:
    # Get the ratios and record them
    ratios = get_ratios(iae_result)
    all_ratios.append(ratios)

# Convert to numpy array for easier analysis
all_ratios_array = np.array(all_ratios, dtype=object)  # Using dtype=object to handle lists of different lengths


## 3 Visualize the ratios

Function to plot the ratios.

Plot for each $\theta$.

In [13]:
def plot_error_ratios(all_ratios, theoretical_ratio = None, i = None, percentile_low=5, percentile_high=95):
    if i is None:
        # Use all rows if i is not provided
        selected_data = all_ratios
        image_name = f'radius_ratio_all_amps_{epsilon_idx + 1}.png'
    else:
        # Calculate the start and end indices for the selected rows
        start_index = i * 1000
        end_index = (i + 1) * 1000
        selected_data = all_ratios[start_index:end_index]
        image_name = f'radius_ratio_{thetas[i]}_{epsilon_idx + 1}.png'  # Naming the image file based on the index

    # Calculate the length of the longest list in selected_data
    max_length = max(len(r) for r in selected_data)

    # Initialize lists to store the avg, stddev, and selected percentiles for each index
    avg_ratios = []
    sd_ratios = []
    percentile_low_values = []
    percentile_high_values = []
    

    # Calculate the avg, stddev, and selected percentiles for each index
    for j in range(max_length):
        current_ratios = [r[j] for r in selected_data if len(r) > j]
        avg_ratios.append(np.mean(current_ratios))
        sd_ratios.append(np.std(current_ratios))
        percentile_low_values.append(np.percentile(current_ratios, percentile_low))
        percentile_high_values.append(np.percentile(current_ratios, percentile_high))

    if i is None:
        print(avg_ratios)
    # Plotting the results
    plt.figure(figsize=(9, 6))
    x_values = range(0, len(avg_ratios))

    # Plot the average ratios
    plt.plot(x_values, [r for r in avg_ratios], 'b-', label='avg.')

    # Plot the shaded area for standard deviation
    plt.fill_between(x_values, 
                     (np.array(avg_ratios) - np.array(sd_ratios)), 
                     (np.array(avg_ratios) + np.array(sd_ratios)), 
                     color='gray', alpha=0.3, label=r'avg. $\pm$ stddev.')

    # Plot the selected percentiles
    plt.plot(x_values, [r for r in percentile_low_values], 'b--', label=f'{percentile_low}th percentile')
    plt.plot(x_values, [r for r in percentile_high_values], 'b:', label=f'{percentile_high}th percentile')

    # Plot reference lines
    if theoretical_ratio is not None:
        plt.axhline(y=np.sqrt(1/theoretical_ratio), color='k', linestyle='--', alpha=0.5, label=r'Theoretical Ratio')

    # Adding labels and title
    plt.xlabel(r't', fontsize=16)
    plt.ylabel(r'Interval Radius Ratio ($\epsilon_{t-1}/\epsilon_{t}$)', fontsize=16)
    plt.ylim(0, 5)
    if i is None:
        plt.title(r'Interval Radius Ratio Paths for All $\theta=0^\circ,5^\circ,\dots,90^\circ$', fontsize=18)
    else:
        plt.title(r'Interval Radius Ratio Paths for $\theta= {}^\circ$'.format(thetas[i]), fontsize=18)
    plt.legend(fontsize=16, ncol=2)
    plt.grid(True)
    
    # Increase font size of tick labels
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    # Create the directory if it doesn't exist
    output_dir = 'figs_radius_ratio'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the figure
    plt.savefig(os.path.join(output_dir, image_name), bbox_inches='tight', dpi = 300)
    # plt.show()

# Example usage:
# plot_error_ratios(all_ratios)  # This will plot all rows and save the image as radius_ratio_all_rows_3.png
# plot_error_ratios(all_ratios, i=0)  # This will plot the first thousand rows and save the image as radius_ratio_0_3.png


In [14]:
for j in range(len(thetas)):
    plot_error_ratios(all_ratios, theoretical_ratio=0.132, i=j)

Plot for all $\theta$.

In [8]:
#plot_error_ratios(all_ratios, 0.132)